# Organización de unidades productivas

## Limpieza de los datos

<b>Objetivo:</b> eliminar las líneas innecesarias, y organizar los encabezados de los documentos. De esta manera podremos unificar todos los datos posteriormente.

In [16]:
import os

def clean_data(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos CSV en la carpeta de entrada
    csv_file = [file for file in os.listdir(input_folder) if file.endswith('.csv')]

    # Datos excepcionales
    unuseful_words = ['Serie;;;2015;2016;2017;2018;2019;2020;2021;2022;2023(A)', 'empresarial']
    headers = "Serie;Codigo_municipio;Nombre;2022;2023\n"

    # Limpiar y guardar los archivos CSV en la carpeta de salida
    for file in csv_file:
        original_file = os.path.join(input_folder, file)
        clean_file = os.path.join(output_folder, file)
        with open(original_file, 'r') as f_input, open(clean_file, 'w') as f_output:
            
            f_output.write(headers) 
            delete_next = False

            for line in f_input:                    
                if not delete_next:
                    delete = False
                    for word in unuseful_words:
                        if word in line or line.strip() == ';':
                            delete = True
                    if line.strip() == '':
                        delete = True
                    elif '(A) Avance' in line:
                        line = line.replace('(A) Avance ;', '')
                    if not delete:
                        fields = line.strip().split(';')
                        f_output.write(';'.join(fields[:3] + fields[-3:]) + '\n')
                    if 'Zarzalejo' in line:
                        delete_next = True

input_folder = ".\original_data"
output_folder = ".\clean_data"

clean_data(input_folder, output_folder)


## Suma de los datos por sector

<b>Objetivo:</b> dividir los archivos por sector y hacer las sumas parciales de cada sector en su propio archivo <i>csv</I>

In [17]:
import os, csv

def sector_divider(input_folder, output_folder):

    # Diccionario para almacenar los datos sumados por sector
    sector_data = {'primario': {}, 'secundario': {}, 'terciario': {}}

    # Obtener la lista de archivos CSV en la carpeta de entrada
    csv_files = os.listdir(input_folder)

    # Mapear cada archivo a su sector correspondiente y sumar los datos
    for file in csv_files:
        if file.endswith('.csv'):

            # Determinar el sector según el nombre del archivo
            if 'agricultura' in file or 'ganaderia' in file or 'pesca' in file:
                sector = 'primario'
            elif 'industria' in file or 'construccion' in file or 'electricidad' in file or 'agua' in file or 'metalurgia' in file:
                sector = 'secundario'
            else:
                sector = 'terciario'

            # Leer el archivo y sumar los datos por sector
            with open(os.path.join(input_folder, file), 'r') as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=';')
                next(csv_reader)

                # Iterar sobre cada fila en el archivo CSV
                for row in csv_reader:
                    serie = row[0] if row else ""
                    codigo_municipio = row[1] if len(row) > 1 else ""
                    nombre = row[2] if len(row) > 2 else ""

                    # Verificar si ya existe una entrada para esta serie y código de municipio
                    key = (serie, codigo_municipio, nombre)
                    if key in sector_data[sector]:
                        for i in range(3, len(row)):
                            sector_data[sector][key][i] += int(row[i]) if row[i] else 0
                    else:
                        sector_data[sector][key] = [serie, codigo_municipio, nombre] + [int(x) if x else 0 for x in row[3:]]

    # Escribir los datos sumados por sector en archivos CSV
    for sector, datos_sumados in sector_data.items():
        output_path = os.path.join(output_folder, f'suma_{sector}.csv')

        with open(output_path, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file, delimiter=';')
            csv_writer.writerow(['Serie', 'Codigo_municipio', 'Nombre', '2022', '2023'])
            for key, values in datos_sumados.items():
                csv_writer.writerow([key[0], key[1], key[2], *values[3:5]])

input_folder = "./clean_data"
output_folder = "./suma_por_sector"
sector_divider(input_folder, output_folder)  

## Suma de los datos por sector

## Unión final de todos los datos en un mismo archivo <i>csv</i>

In [18]:
import os
import csv
import re

def join_data(folder_path, output_path):
    join_data = {}
    csv_files = sorted(os.listdir(folder_path))

    # Iterar sobre cada archivo en la carpeta
    for file in csv_files:
        if file.endswith('.csv'):
            with open(os.path.join(folder_path, file), 'r') as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=';')
                next(csv_reader)
                sector = file.split('.')[0].split('_')[1]

                for row in csv_reader:
                    serie = row[0] if row else ""
                    codigo_municipio = row[1] if len(row) > 1 else ""
                    nombre = row[2] if len(row) > 2 else ""

                    # Verificar si la serie, código de municipio y nombre ya están en el diccionario
                    if (serie, codigo_municipio, nombre) not in join_data:
                        join_data[(serie, codigo_municipio, nombre)] = {}

                    # Obtener los datos para cada intervalo de edad
                    for i, valor in enumerate(row[3:5], start=2022):
                        clave = f"unidades_productivas_{sector}_{i}"
                        join_data[(serie, codigo_municipio, nombre)][clave] = int(valor) if valor else 0

    # Escribir los datos sumados en un nuevo file CSV
    with open(output_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';')
        # Escribir los encabezados en el file CSV
        final_keys = []

        for key, values in join_data.items():
            if final_keys == []:
                final_keys.extend(values.keys())
        final_keys = [str(clave) for clave in final_keys]

        # Escribir los encabezados en el file CSV
        csv_writer.writerow(['Serie', 'Codigo_municipio', 'Nombre', *final_keys])

        # Escribir los datos sumados
        for key, values in join_data.items():
            line = [key[0], key[1], key[2]]
            for clave in final_keys:
                line.append(values.get(clave, '0'))
            csv_writer.writerow(line)

folder_path = './suma_por_sector'
output_path = '../union_final/unidades_productivas.csv'

join_data(folder_path, output_path)